In [ ]:
import pandas as pd
from shapely.geometry import Point
import numpy as np


#import geopy.distance
from geopy.distance import vincenty

from geographiclib.geodesic import Geodesic
import csv

import matplotlib.colors as mcolors
from numpy import arctan2,random,sin,cos,degrees
import seaborn as sn
import matplotlib.pyplot as plt
from matplotlib.pyplot import subplot, figure

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
#    Find Azimuth vs Distance user distribution

In [ ]:
# Step 1 create plotfile.csv to visualize the graph.


celld=pd.read_csv("cell_det.csv") #cell details file - contain global cell id, cell longitude, cell latitude, cell azimuth etc
userd=pd.read_csv("userloc.csv") #user gps details, related global cell id, use logi & lati.,  

cellDframe=celld[['global_cellid','Azimuth','longitude','latitude']] #get related details for future calculations from cell details csv
userDframe=userd[['serving_cell_id','serving_cell_rsrp','serving_cell_rsrq','user_latitude','user_longitude']] #get related details for fuure calculations from user details csv


outframe=[["GlobalCellid","CellAzimuth","UserDistance","UserAzimuth",'cellLati','cellLongi','UserLati','UserLongi',"RSRQ","RSRP"]] # output data structure. this is a 2d array(this is header row/column names). collumn names defined




for i,row in cellDframe.iterrows():  # select row from cell details df
  gcid=row['global_cellid']
  tempudf=userDframe[userDframe["serving_cell_id"]==gcid] # tempory df from user details related to selected global cell id.  
  cell_lon=row['longitude']
  cell_lat=row['latitude']
  cazi=row['Azimuth']
  pos2=(cell_lat, cell_lon) #define position for the cell

  for j,row2 in tempudf.iterrows():#select each row from temory df of user details. a row has details for one user



    pos1=(row2['user_latitude'],row2['user_longitude']) #define posion array for selected user from the row
    dis=vincenty(pos1,pos2).m #get diatance between cell and user
    ubrng=Geodesic.WGS84.Inverse(cell_lat, cell_lon,row2['user_latitude'], row2['user_longitude'])['azi1'] # get bearing between user and cell()

    if(ubrng<0):#convert bearing to azimuth if bearing<0 ; bearing have positive value, use right or left rotation from north. therefore we need to add 360 degree to get azimuth for negetive bearing
      ubrng+=360

    rsrq=row2['serving_cell_rsrq']
    rsrp=row2['serving_cell_rsrp']

    outline=[gcid,cazi,dis,ubrng,cell_lat,cell_lon,row2['user_latitude'],row2['user_longitude'],rsrq,rsrp] # define user details for plot

    outframe.append(outline) # append to output 2d list


# list->>csv

with open('plotfile.csv','w',newline='') as myoutput:
  writer=csv.writer(myoutput)
  writer.writerows(outframe)
       







  #tempudf['distance']=tempudf.apply(lambda x: vincenty((x[3],x[4]),pos2))




In [ ]:
   # Draw user distribution on the cartesian azimuth vs distance

In [ ]:
plotdfA=pd.read_csv("plotfile.csv")


#plotdfB=plotdfA[plotdfA["UserDistance"]>1000]

cellids=plotdfA['GlobalCellid'].unique() # cell id list

cellid=cellids[10] # select a cellid from list to plot

plotdf=plotdfA[plotdfA['GlobalCellid']==cellid] # get user df related to selected cell id
plotdf=plotdf[plotdf["UserDistance"]<1000] # remove outliers


y1=plotdf['UserDistance'] # define axis
x1=plotdf['UserAzimuth']

# draw figure

plt.figure(figsize=(100,100),frameon=True,edgecolor='white')

# plot user locations

plt.plot(x1,y1, label = "Dist_vs_Azim(user)", color="white",linestyle='', linewidth = 0.5, 
         marker='.', markerfacecolor='blue', markersize=20)


# plot cell azimuth line
x2=plotdf["CellAzimuth"]
y2=plotdf['UserDistance']
plt.plot(x2,y2,label='cell_Azimuth',color='red',linestyle='-',linewidth=2,marker='|',markersize=30)


# plot rsrp destribution vs user azimuth
y3=plotdf["RSRP"]
x3=plotdf['UserAzimuth']
plt.plot(x3,y3,label='RSRP',color='black',linestyle='',linewidth=0.5,marker='1',markersize=10,markerfacecolor='black')


#plot rsrq destribution vs user azimuth
y3=plotdf["RSRQ"]
x3=plotdf['UserAzimuth']
plt.plot(x3,y3,label='RSRQ',color='green',linestyle='',linewidth=0.5,marker='2',markersize=10,markerfacecolor='green')


plt.ylabel('y- axis/Distance-RSRP-RSRQ',fontsize=30)
# naming the y axis 
plt.xlabel('x - axis/Cell and User Azimuth',fontsize=30) 
# giving a title to my graph 
#plt.title('Azimuth vs Distance and Azimuth of cell',fontsize=50)
#plt.suptitle('Drop_Outliers version', fontsize=45) 
dris="global_cellid  : "+str(cellid)+"  |  Cell_Azimeth : "+str(plotdf["CellAzimuth"].unique())+"filtered Outliers"
plt.suptitle(dris, fontsize=80) 

# show a legend on the plot 
plt.legend() 

# function to show the plot
plt.show() 




In [ ]:
# Grid Analysis

In [ ]:

#read csvs
plotdf=pd.read_csv("plotfile.csv")
cellDframe=pd.read_csv('cell_det.csv')




# plot df colhumns [gcid,cazi,dis,user azimuth,rsrq,rsrp]       "GlobalCellid","CellAzimuth","UserDistance","UserAzimuth","RSRQ","RSRP"

#remove outliers

plotdf=plotdf[plotdf['UserDistance']<1000]


#get universal mean stds for rp and rq then set bounderies for seperating bins

rpstd=plotdf['RSRP'].std()
rpmean=plotdf['RSRP'].mean()

rqstd=plotdf['RSRQ'].std()
rqmean=plotdf['RSRQ'].mean()

rplowlim1=rpmean-rpstd
rplowlim2=rpmean-2*rpstd

rpuplim1=rpmean-rpstd
rpuplim2=rpmean-2*rpstd



rqlowlim1=rqmean-rqstd
rqlowlim2=rqmean-2*rqstd

rquplim1=rqmean-rqstd
rquplim2=rqmean-2*rqstd



In [ ]:


plotdf['UDCategory']=pd.cut(plotdf['UserDistance'], bins=np.arange(0, 1001, 10),labels=np.arange(0,1000,10).tolist()) # create bins for user distance 
plotdf['UACategory']=pd.cut(plotdf['UserAzimuth'], bins=np.arange(0, 361, 5),labels=np.arange(0,360,5).tolist()) #create bis for user azimuth

#necessary convertion of data types to future purposes

plotdf['UACategory']=plotdf['UACategory'].astype('int')
plotdf['UDCategory']=plotdf['UDCategory'].astype('int')
plotdf['GlobalCellid']=plotdf['GlobalCellid'].astype('int')

In [ ]:
#define functions for lambda function - get rsrp color and rsrq color for each rsrp, rsrq bin


def rpcolor(rpval):
  if (rpval<rplowlim2):
    return 1  #brown

  elif (rpval<rplowlim1):
    return 2 #red
  elif (rpval<rpuplim1):
    return 3 #Orange
  elif (rpval<rpuplim2):
    return 4 #gold
  else:
    return 5 #@ yellow
  

def rqcolor(rqval):
  if (rqval<rqlowlim2):
    return 1 #Olive

  elif (rqval<rqlowlim1):
    return 2 #lime
  elif (rqval<rquplim1):
    return 3 #blue
  elif (rqval<rquplim2):
    return 4 #violet
  else:
    return 5 #pink



# apply function to df get related strength bin rsrp & rsrq
plotdf['RSRPColor']= plotdf.apply(lambda x: rpcolor(x['RSRP']),axis=1)
plotdf['RSRQColor']= plotdf.apply(lambda x: rqcolor(x['RSRQ']),axis=1)


# set  centers of each bins user distance and user azimuth for get the center of cells of grid of graph

plotdf['Xcenter']=plotdf['UACategory']+2.5
plotdf['Ycenter']=plotdf['UDCategory']+5



In [ ]:
#D I S T R I B U T I O N P L O T - Mean of rsrp rsrq for grid    categorycal rsrp, rsrq version      ****ON SAME XY CARTESIAN &&& MARKER X and O

In [ ]:
for i,row in cellDframe.iterrows(): # iteration rows on cell details df
  gcid=row['global_cellid'] #select a cell from cell details df
  azim=row['Azimuth'] # get cell azimuth for selected one
  tempudf=plotdf[plotdf["GlobalCellid"]==gcid] # make temp df from related user df 
  gbdf = tempudf.groupby(['Xcenter','Ycenter'],as_index=False)[['RSRP','RSRQ']].agg(['mean','std']).reset_index() # create the grids using group by user distance and azimuth categories - aggregations give mean and sts of rsrp & rsrq for each grid
  gbdf.fillna(0,inplace=True) # fill empty grids with 0 - no users avalable for that locations

  gbdf.columns = ['_'.join(col) for col in gbdf.columns] # join the two column headers - multi index to one level. usefull when apply lambda function
  gbdf['RSRPColor']= gbdf.apply(lambda x: rpcolor(x['RSRP_mean']),axis=1) # apply lambda function to rsrp to get grid color
  gbdf['RSRQColor']= gbdf.apply(lambda x: rqcolor(x['RSRQ_mean']),axis=1) # same for rsrq


 # draw figure

  plt.ioff()
  f,(ax1,ax2)=plt.subplots(2)
  f.set_figheight(150)
  f.set_figwidth(80)
  title=str(gcid)+' AZIMUTH '+str(azim)

  f.suptitle(title, fontsize=40)


  ##x_data=datadf.iloc[:,1].tolist()
  #y_data=datadf.iloc[:,1410].tolist()

  #RSRP
  rp = ax1.scatter('Xcenter_', 'Ycenter_', 
              s=2500,
              alpha=1,c='RSRPColor',edgecolors='Black',linewidth=3,
              data=gbdf)
  ax1.axvline(azim, color='r')
  #[plt.legend(loc='Upper right')


  rq = ax2.scatter('Xcenter_', 'Ycenter_', 
              s=2000,marker='X',
              alpha=1,c='RSRQColor',edgecolors='Black',linewidth=3,
              data=gbdf)
  ax2.axvline(azim, color='r')


  #legend1 = x.legend(*rp.legend_elements(),
  #                    loc="upper left", title="RSRP Strength 1 to 4 SYMBOL |O|")
  #x.add_artist(legend1)

  #legend2 = x.legend(*rq.legend_elements(),
  #                   loc="upper right", title="RSRQ Strength 10 to 14 SYMBOL |X|")
  #x.add_artist(legend2)


  #x.xlabel("X", size=16)
  #x.ylabel("y", size=16)
  #x.title("Bubble Plot with Matplotlib", size=18)


  plt.show()

  #set save location
  fname='/content/drive/My Drive/10m5dgree/'+str(gcid)+'.jpg'
  f.savefig(fname)



  

In [ ]:
# D IS T R I B U T I O N P L O T -----------      Color gradient version  *****TWO SUBPLOTS

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable
plt.rc('axes', labelsize=30)
plt.rc('xtick', labelsize=25)    # fontsize of the tick labels
plt.rc('ytick', labelsize=25)    # fontsize of the tick labels
plt.rc('legend', fontsize=25)
for i,row in cellDframe.iterrows(): # iteration rows on cell details df
  gcid=row['global_cellid'] #select a cell from cell details df
  azim=row['Azimuth'] # get cell azimuth for selected one
  tempudf=plotdf[plotdf["GlobalCellid"]==gcid] # make temp df from related user df 
  gbdf = tempudf.groupby(['Xcenter','Ycenter'],as_index=False)[['RSRP','RSRQ']].agg(['mean','std']).reset_index() # create the grids using group by user distance and azimuth categories - aggregations give mean and sts of rsrp & rsrq for each grid
  gbdf.fillna(0,inplace=True) # fill empty grids with 0 - no users avalable for that locations

  gbdf.columns = ['_'.join(col) for col in gbdf.columns] # join the two column headers - multi index to one level. usefull when apply lambda function
  #gbdf['RSRPColor']= gbdf.apply(lambda x: rpcolor(x['RSRP_mean']),axis=1) # apply lambda function to rsrp to get grid color
  #gbdf['RSRQColor']= gbdf.apply(lambda x: rqcolor(x['RSRQ_mean']),axis=1) # same for rsrq


 # draw figure

  plt.ioff()
  f,(ax1,ax2)=plt.subplots(2)
  f.patch.set_facecolor('white')
  f.set_figheight(250)
  f.set_figwidth(100)
  title=str(gcid)+' AZIMUTH '+str(azim)
  f.suptitle(title, fontsize=40)


  ##x_data=datadf.iloc[:,1].tolist()
  #y_data=datadf.iloc[:,1410].tolist()

  #RSRP



  ax1.set_title("RSRP Strength Distribution")
  rp = ax1.scatter('Xcenter_', 'Ycenter_', 
              s=2500,marker='s',
              alpha=1,c='RSRP_mean',edgecolors='Black',linewidth=3,cmap='winter',
              data=gbdf)
  #im = ax1.imshow(gbdf['RSRP_mean'], cmap='winter')
  ax1.set_xlabel('Distance/m')
  ax2.set_ylabel('Azimuth/degrees')
  divider = make_axes_locatable(ax1)
  cax = divider.append_axes('right', size='2%', pad=0.1)
  #cbar=plt.colorbar(ax=ax1)
  cbr=plt.colorbar(rp, cax=cax, orientation='vertical')
  cbr.set_label('RSRP_mean')
  ax1.axvline(azim, color='r')
  #[plt.legend(loc='Upper right')

  ax2.set_title("RSRQ Strength Distribution")
  rq = ax2.scatter('Xcenter_', 'Ycenter_', 
              s=2500,marker='s',
              alpha=1,c='RSRQ_mean',edgecolors='Black',linewidth=3,cmap='spring',
              data=gbdf)
  #im = ax2.imshow(gbdf['RSRQ_mean'], cmap='spring')
  ax2.set_xlabel('Distance/m')
  ax2.set_ylabel('Azimuth/Degrees')
  divider = make_axes_locatable(ax2)
  cax = divider.append_axes('right', size='2%', pad=0.1)
  #cbar2=plt.colorbar(ax=ax2)
  cbr2=plt.colorbar(rq, cax=cax, orientation='vertical')
  cbr2.set_label('RSRQ_mean')
  
  ax2.axvline(azim, color='r')




  #plt.show()
  #set save location
  fname='/content/drive/My Drive/10m5dgree/'+str(gcid)+'.jpg'
  f.savefig(fname)
  plt.close(f)



  